In [ ]:
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

In [1]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(f"TensorFlow Version: {tf.__version__}")

# --- 设置 T4 x2 分布式策略 ---
# 这会利用您的两个GPU
try:
    strategy = tf.distribute.MirroredStrategy()
    print('已启用 MirroredStrategy')
    print('  可用的设备数量: {}'.format(strategy.num_replicas_in_sync))
except ValueError:
    print("未检测到 MirroredStrategy")
    strategy = tf.distribute.get_strategy() # 默认为单个设备

2025-11-09 20:51:39.881413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762721499.905263     126 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762721499.912609     126 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow Version: 2.18.0
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
已启用 MirroredStrategy
  可用的设备数量: 2


I0000 00:00:1762721506.408663     126 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1762721506.409441     126 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [2]:
# --- 加载预处理文件 ---
ENCODERS_PATH = '/kaggle/input/csiro-data-preprocessing/encoders.pkl'

try:
    with open(ENCODERS_PATH, 'rb') as f:
        encoders = pickle.load(f)
    
    # 获取嵌入层的 'input_dim' (类别总数)
    N_STATES = len(encoders['State'].classes_)
    N_SPECIES = len(encoders['Species'].classes_)
    
    print(f"加载 'State' 编码器: {N_STATES} 个唯一州")
    print(f"加载 'Species' 编码器: {N_SPECIES} 个唯一种类")

except FileNotFoundError:
    print(f"错误: 无法在 {ENCODERS_PATH} 找到编码器文件。")
    # 设置占位符以便代码可以运行，但您必须修复路径
    N_STATES, N_SPECIES = 4, 15 

# --- 模型常量 ---
# 基于您的EDA (2000x1000 -> 2:1 比例)
# 384x192 是一个很好的平衡点
IMG_HEIGHT = 384
IMG_WIDTH = 192
IMG_CHANNELS = 3

# 嵌入层的输出维度 (超参数)
STATE_EMBED_DIM = 4
SPECIES_EMBED_DIM = 8

加载 'State' 编码器: 4 个唯一州
加载 'Species' 编码器: 15 个唯一种类


In [3]:
def build_model(n_states, n_species, img_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    """
    构建双分支多输入融合网络
    """
    
    # --- 1. 定义所有输入 ---
    
    # 分支1 输入
    image_input = layers.Input(shape=img_shape, name="image_input")
    
    # 分支2 输入
    # 我们将数值、时间和类别特征分开
    num_input = layers.Input(shape=(2,), name="num_input")       # NDVI, Height
    time_input = layers.Input(shape=(2,), name="time_input")      # month_sin, month_cos
    state_input = layers.Input(shape=(1,), name="state_input")    # State (整数)
    species_input = layers.Input(shape=(1,), name="species_input") # Species (整数)
    
    
    # --- 2. 构建分支 1: 图像 (CNN) ---
    print("构建分支 1: 图像 (EfficientNetB4)...")
    # 使用 EfficientNetB4，如方案所述
    base_model = tf.keras.applications.EfficientNetB4(
        include_top=False,
        weights='imagenet',
        input_tensor=image_input
    )
    # 保持模型可训练
    base_model.trainable = True
    
    # 头部
    x_img = layers.GlobalAveragePooling2D(name="img_gap")(base_model.output)
    x_img = layers.BatchNormalization()(x_img)
    x_img = layers.Dense(1024, activation='relu', name="img_dense_1024")(x_img)
    
    
    # --- 3. 构建分支 2: 表格 (MLP) ---
    print("构建分支 2: 表格 (MLP)...")
    
    # 嵌入层
    state_embed = layers.Embedding(
        input_dim=n_states, 
        output_dim=STATE_EMBED_DIM, 
        name="state_embedding"
    )(state_input)
    species_embed = layers.Embedding(
        input_dim=n_species, 
        output_dim=SPECIES_EMBED_DIM, 
        name="species_embedding"
    )(species_input)
    
    # 展平嵌入层
    state_embed = layers.Flatten()(state_embed)
    species_embed = layers.Flatten()(species_embed)
    
    # 拼接所有表格特征
    x_tab = layers.Concatenate(name="tab_concat")([
        num_input,      # 2 a
        time_input,     # 2 a
        state_embed,    # 4 a
        species_embed   # 8 a
    ]) # 总共 2 + 2 + 4 + 8 = 16 个特征
    
    # MLP 头部 (如方案所述)
    x_tab = layers.Dense(128, activation='relu', name="tab_dense_128")(x_tab)
    x_tab = layers.BatchNormalization()(x_tab)
    x_tab = layers.Dropout(0.3)(x_tab)
    x_tab = layers.Dense(64, activation='relu', name="tab_dense_64")(x_tab)
    
    
    # --- 4. 融合与最终头部 ---
    print("融合分支并构建最终头部...")
    
    # 融合两个分支的输出
    fused = layers.Concatenate(name="fusion_concat")([x_img, x_tab])
    fused = layers.BatchNormalization()(fused)
    fused = layers.Dropout(0.4)(fused)
    
    # 最终的MLP头部
    fused = layers.Dense(512, activation='relu', name="head_dense_512")(fused)
    fused = layers.BatchNormalization()(fused)
    
    # 最终输出层 (如方案所述)
    # 4个神经元, 'linear' 激活 (因为我们预测的是对数值)
    output = layers.Dense(4, activation='linear', name="output")(fused)
    
    
    # --- 5. 创建并返回模型 ---
    model = tf.keras.Model(
        inputs=[image_input, num_input, time_input, state_input, species_input],
        outputs=output,
        name="Image2Biomass_Model"
    )
    
    return model

In [5]:
# --- 1. 打印模型概况 ---
model.summary()

# --- 2. 定义模型保存路径 ---
# 我们使用 .keras 格式，这是推荐的格式
MODEL_PATH = '/kaggle/working/image2biomass_model.keras'

# --- 3. 保存完整的模型 ---
print(f"\n正在将完整模型保存到: {MODEL_PATH} ...")
try:
    # model.save() 会保存架构、权重和优化器状态
    model.save(MODEL_PATH)
    
    print(f"模型已成功保存。")
    
except Exception as e:
    print(f"保存模型时出错: {e}")

Model: "Image2Biomass_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 384, 192,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 384, 192,  │          0 │ image_input[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 384, 192,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 384, 192,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 385, 193,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 192, 96,   │      1,296 │ stem_conv_pad[0]… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 192, 96,   │        192 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 192, 96,   │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 192, 96,   │        432 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 192, 96,   │        192 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 192, 96,   │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 48)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 48)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 12)  │        588 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 48)  │        624 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 192, 96,   │          0 │ block1a_activati… │
│ (Multiply)          │ 48)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 192, 96,   │      1,152 │ block1a_se_excit

 Total params: 20,094,123 (76.65 MB)

 Trainable params: 19,961,876 (76.15 MB)

 Non-trainable params: 132,247 (516.59 KB)


正在将完整模型保存到: /kaggle/working/image2biomass_model.keras ...
模型已成功保存。
